In [1]:
import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import re
import pandas_datareader.data as web
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

from scrape_benzinga import get_initial_biggest_movers_data, get_new_benzinga_data, get_initial_price_data, get_earnings_dates, get_new_earnings_dates,get_new_price_data
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Note: put %%capture at the start of each cell to suppress printed output

The steps below are for updating already existing headline, price, volume, and earnings dates data:

In [2]:
#old benzinga data
old_dat = pd.read_csv('stock_headlines.csv')

#old price and volume data
prices = pd.read_csv('stock_prices.csv')
volumes = pd.read_csv('stock_volumes.csv')

#old earnings data
earnings = pd.read_csv('filing_dates.csv')

In [3]:
%%capture
#get new benzinga headlines
new_dat = get_new_benzinga_data(old_dat)

In [4]:
#concat old and new headlines
df = pd.concat([new_dat, old_dat], axis = 0)

In [5]:
%%capture
#get updated prices and volumes
new_prices, new_volumes = get_new_price_data(set(df.ticker.values), prices)

In [6]:
#concat to old prices and volumes
prices = pd.concat([prices, new_prices], axis = 0)
volumes = pd.concat([volumes, new_volumes], axis = 0)

#sort by ticker then by date
prices = prices.sort_values(['ticker', 'date'], ascending = True)
volumes = volumes.sort_values(['ticker', 'date'], ascending = True)

prices = prices.drop_duplicates()
volumes = volumes.drop_duplicates()

In [7]:
#only keep headlines where we could get price data for
df2 = df[df.ticker.isin(set(prices.ticker))]

In [8]:
%%capture
#get updated earning filing dates
new_earnings = get_new_earnings_dates(set(df.ticker.values), earnings, '2015-10-12')

In [14]:
#concat new earnings to old earnings
earnings = pd.concat([new_earnings, earnings], axis = 0)
earnings = earnings.sort_values(['ticker', 'filing_dates'], ascending = True)
earnings = earnings.drop_duplicates()

In [15]:
#save stock headlines and price data
df2.to_csv('stock_headlines.csv', index = False)
volumes.to_csv('stock_volumes.csv', index = False)
prices.to_csv('stock_prices.csv', index = False)
earnings.to_csv('filing_dates.csv', index = False)

The cells below are for doing an initial pull of headline, price, volume, and earnings date data. Benzinga scraper function works for articles going back to October 12, 2015.

In [16]:
%%capture
#get biggest movers data
new = get_initial_biggest_movers_data('October 12, 2015')

In [17]:
%%capture
#get initial price and volume data
tcks = set(df.ticker.tolist())
prices, volumes = get_initial_price_data(tcks, '2015-10-01')

In [18]:
#only keep headlines where we could get price data for
df2 = df[df.ticker.isin(set(prices.ticker))]

In [19]:
%%capture
#get earning filing dates
earnings = get_earnings_dates(set(df2.ticker), '2015-10-09')

In [10]:
#save stock headlines and price data
df2.to_csv('stock_headlines.csv', index = False)
volumes.to_csv('stock_volumes.csv', index = False)
prices.to_csv('stock_prices.csv', index = False)
earnings.to_csv('filing_dates.csv', index = False)